In [ ]:
import pandas as pd
import math
import numpy as np
import datetime

import matplotlib.pyplot as plt

import time
import random

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

from sklearn.metrics import r2_score

import pandas_datareader.data as web
import pprint
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import matplotlib.dates as mdates

%matplotlib inline

# from genetic_selection import GeneticSelectionCV   
# I have phone verification issue. So i can't import this module

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = "1_71asScU8XI31EIcWF_ikPj0ihvcgfOu"  
file = drive.CreateFile({'id':id}) 
file.GetContentFile('DOW20200323') 

id = "1raZfnsw8fsxxHQ5e5oFIM-zmmXiiWKCX"  
file = drive.CreateFile({'id':id}) 
file.GetContentFile('DOW20200324') 

In [ ]:
import pandas as pd

dow_324 = pd.read_csv('DOW20200324')
dow_323 = pd.read_csv('DOW20200323')
dow_323.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78070 entries, 0 to 78069
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   symbol          78070 non-null  object 
 1   timestamp       78070 non-null  object 
 2   bid_price       74723 non-null  float64
 3   bid_size        74723 non-null  float64
 4   bid_price_lag   74723 non-null  float64
 5   bid_size_lag    74723 non-null  float64
 6   V_of_B          74723 non-null  float64
 7   ask_price       74723 non-null  float64
 8   ask_size        74723 non-null  float64
 9   ask_price_lag   74723 non-null  float64
 10  ask_size_lag    74723 non-null  float64
 11  V_of_A          74723 non-null  float64
 12  VOI             74723 non-null  float64
 13  OIR             74723 non-null  float64
 14  bid_ask_spread  74723 non-null  float64
 15  VWAP_max        77455 non-null  float64
 16  VWAP_mean       77455 non-null  float64
 17  VWAP_min        77455 non-null 

In [ ]:
random.seed(30)
# Get the dataframe with the corresponding x and y
def read_stock(df, ticker="AAPL", n_period=1, n_shift=-1):

  df_ =df.copy()
  dow_df = df_.replace([np.inf, -np.inf], np.nan)
  dow_df = dow_df.dropna()
  df_ticker = dow_df[dow_df['symbol'] == ticker]
  df_ticker_ = df_ticker.loc[:, ["timestamp", "VOI", "OIR", "bid_ask_spread", "EWM", "OBV"]]
  df_ticker_["return"] = (df_ticker["VWAP_mean"].pct_change(periods = n_period).shift(n_shift))
  df_ticker_= df_ticker_[:-1] # Delete the missing value
  
  df_ticker_.set_index('timestamp', inplace=True) 
  return df_ticker_

In [ ]:
df_23 = read_stock(dow_323, "WMT", 30, -1)
df_24 = read_stock(dow_324, "WMT", 30, -1)

In [ ]:
df_23.isnull().sum()

VOI                0
OIR                0
bid_ask_spread     0
EWM                0
OBV                0
return            29
dtype: int64

In [ ]:
random.seed(30)
# Basic models function
def functions(df_X, model="LinearRegression"): #(model="LinearRegression", "SVR")

  df_ = df_X.copy()
  norm_train_data = df_.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]] #scale df_previousDay
  #norm_test_data = scaleData(df_previousDay, df_nextDay)[1] #scale df_nextDay
  norm_train_target = df_["return"][:, None] #y


  from sklearn.linear_model import LinearRegression
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.linear_model import ElasticNet
  from sklearn.svm import SVR

  if model == "LinearRegression":
    reg = LinearRegression()
    reg.fit(norm_train_data, norm_train_target)
  
  elif model == "SVR":
    reg = SVR(C=1, gamma = 0.001, kernel='poly')
    reg.fit(norm_train_data, norm_train_target)
  
  elif model == "ElasticNet":
    reg = ElasticNet(alpha=1, l1_ratio=0.5, tol=0.0001)
    reg.fit(norm_train_data, norm_train_target)

  elif model == "RandomForestRegressor":
    reg = RandomForestRegressor(max_depth=100, min_samples_leaf=2, min_samples_split=5)
    reg.fit(norm_train_data, norm_train_target)
  else:
    return "Error"
  
  return reg

In [ ]:
def RF_crossval(df_Day, n_periods=1):
    """
    Wrap Random Forest for reuse
    price is the dataframe
    periods is the percent change differences between time intervals. Default is 1 (minute)
    """
    #drop blank
    price_rf =df_Day.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV", "VWAP_mean"]]
    
    #create DV. Rising is 1, unchanged or goes down is 0
    price_rf["target"]=pd.Series()
    price_rf.loc[price_rf["VWAP_mean"].pct_change(periods =n_periods)>0,"target"]=1 #Buy
    price_rf.loc[price_rf["VWAP_mean"].pct_change(periods =n_periods)<=0,"target"]=0# Sell

    
    #change infinity to N/a and drop them
    price_rf = price_rf.replace([np.inf, -np.inf], np.nan)
    time.sleep(1)
    price_rf = price_rf.dropna()
    
    target = price_rf["target"].astype("category")
    features = price_rf.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]]
  #df_ticker_["return"] = (df_ticker["VWAP_mean"].pct_change().shift(shift))
    #create crossvalidation
    
    return features,target 

In [ ]:
RF_crossval(dow_324, n_periods=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


(          VOI       OIR  bid_ask_spread         EWM    OBV
 380     100.0  0.000000            0.18  223.610192  -49.0
 381       0.0  0.722607           -0.07  227.919924  103.1
 382       0.0 -1.000000            0.18  230.984298    1.0
 383   -2400.0 -0.923077           -0.08  232.104885  651.0
 384       0.0  0.000000           -0.14  233.300355   51.0
 ...       ...       ...             ...         ...    ...
 78224     0.0  0.000000           55.00   37.220446    1.0
 78225     0.0 -0.500000           55.00   37.220446    1.0
 78226     0.0 -1.000000           55.00   37.220446    1.0
 78227     0.0 -1.000000           55.00   37.220446    1.0
 78228   100.0  1.000000           52.73   35.340002  -49.0
 
 [74806 rows x 5 columns], 380      0.0
 381      1.0
 382      1.0
 383      1.0
 384      1.0
         ... 
 78224    0.0
 78225    0.0
 78226    0.0
 78227    0.0
 78228    0.0
 Name: target, Length: 74806, dtype: category
 Categories (2, float64): [0.0, 1.0])

--------

In [ ]:
def ensemble_table(df, df_P, ticker = 'WMT', model="LinearRegression"):  # df_previousDay, df_nextDay,

  from sklearn import metrics
  from math import sqrt
  import sys



  period_list = {
    "10 s": 1,
    '1 minutes':6,
    '5 minutes':30
}

  results_col=["periods_name","periods","RMSE", "r2", 
              "mse", "accuracy_score"
            ]
  results = pd.DataFrame(columns = results_col)


  for key, value in period_list.items():
    #use value in the dictionary for the period
        
    #norm_train_data,norm_train_target = RF_crossval(df_previousDay,n_periods = value)
    #print("*"*100)
    #print("Now running....periods: ",key)
# CNN 

    #cnn_model=create_cnn(norm_train_data, norm_train_target, activation_function="relu")
    #get 23 model
    get_df = read_stock(df_P, ticker= ticker, n_period= value, n_shift=-1)
    get_df.dropna(how = "any", inplace = True)
    cnn_model=functions(get_df, model)
    
    get_df24 = read_stock(df, ticker= ticker, n_period= value, n_shift=-1)
    get_df24.dropna(how = "any", inplace = True)

    norm_test_data = get_df24.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]] #scale df_nextDay
    norm_test_target = get_df24["return"][:, None]


    score = [sqrt(metrics.mean_squared_error(cnn_model.predict(norm_test_data), norm_test_target)), metrics.r2_score(cnn_model.predict(norm_test_data), norm_test_target)]
    rf_result_list = [key,value,score[0],score[1]]
    
    eval_result_list = rf_result_list[:]

    # #-----------------------------------
    X_test, y_test = RF_crossval(df,n_periods = value) #manipulate data for deep learning
    predicted_y = cnn_model.predict(X_test) #prediction
    predicted_y = np.where(np.array(predicted_y) > 0, 1, 0)
    #print("Prediction results using Deep Learning is....")
    #print(metrics.mean_squared_error(y_test,predicted_y))

  
    # #add evaluation metrics to row
    # a = metrics.precision_recall_fscore_support(y_test,predicted_y)
    # for b in a:
    #       [eval_result_list.append(q) for q in b]
    #besides precision, recall, and fscore, append other metrics
    eval_result_list.append(metrics.mean_squared_error(y_test,predicted_y))
    eval_result_list.append(metrics.accuracy_score(y_test,predicted_y))
    
  
    # # #add row to data frame
    result = pd.DataFrame([eval_result_list],columns = results_col)
    results = results.append(result,ignore_index=True)
    # # # #empty out eval_result_list
  return results 

In [ ]:
ensemble_table(dow_324, dow_323, model="LinearRegression")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy 

,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.000371,-522.003000,0.418442,0.581558
1,1 minutes,6,0.000946,-42.925573,0.472034,0.527966
2,5 minutes,30,0.001681,-5.904082,0.487100,0.512900


In [ ]:
ensemble_table(dow_324, dow_323, model="ElasticNet")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy 

,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.000371,0.000000e+00,0.226733,0.773267
1,1 minutes,6,0.000950,-3.141519e+35,0.451849,0.548151
2,5 minutes,30,0.001677,0.000000e+00,0.582279,0.417721


In [ ]:
ensemble_table(dow_324, dow_323, model="SVR")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is 

,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.003470,0.0,0.773267,0.226733
1,1 minutes,6,0.003255,0.0,0.548151,0.451849
2,5 minutes,30,0.002784,0.0,0.417721,0.582279


In [ ]:
ensemble_table(dow_324, dow_323, model="RandomForestRegressor")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_pa

,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.000371,-1.048287e+06,0.226747,0.773253
1,1 minutes,6,0.000940,-4.134482e+02,0.481766,0.518234
2,5 minutes,30,0.001642,-8.319044e+00,0.480082,0.519918


## cnn model

In [ ]:
def create_cnn(df_P, n_periods=1, activation_function="relu"):


  # norm_train_data = np.expand_dims(RF_crossval(df,n_periods=1)[0], axis=-1)
  # norm_train_target = RF_crossval(df,n_periods=1)[1]
  norm_train_data = df_P.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]] #scale df_previousDay
  norm_train_data = np.expand_dims(norm_train_data, axis=-1)
  #norm_test_data = scaleData(df_previousDay, df_nextDay)[1] #scale df_nextDay
  norm_train_target = df_P["return"][:, None]


  from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
  model = Sequential()
  model.add(Conv1D(50, kernel_size=1, activation=activation_function, input_shape=(5,1)))
  model.add(MaxPooling1D())
  model.add(Conv1D(20, kernel_size=1, activation=activation_function))
  model.add(MaxPooling1D())
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(1, activation=activation_function))
  model.compile(loss='mse', optimizer='SGD', metrics=['accuracy'])  #'rmsprop'
  # Fit 323 data
  model.fit(norm_train_data, norm_train_target, 
                    epochs=10, batch_size=100)

  return model

In [ ]:
#create_cnn(get_df, activation_function="relu")

In [ ]:
def cnn_table(df, df_P, ticker = 'WMT', afunc="relu"):  # df_previousDay, df_nextDay,

  from sklearn import metrics
  from math import sqrt
  import sys



  period_list = {
    "10 s": 1,
    '1 minutes':6,
    '5 minutes':30
}

  results_col=["periods_name","periods","RMSE", "r2", 
              "mse", "accuracy_score"
            ]
  results = pd.DataFrame(columns = results_col)


  for key, value in period_list.items():
    #use value in the dictionary for the period
        
    #norm_train_data,norm_train_target = RF_crossval(df_previousDay,n_periods = value)
    #print("*"*100)
    #print("Now running....periods: ",key)
# CNN 

    #cnn_model=create_cnn(norm_train_data, norm_train_target, activation_function="relu")
    #get 23 model
    get_df = read_stock(df_P, ticker= ticker, n_period= value, n_shift=-1)
    get_df.dropna(how = "any", inplace = True)
    cnn_model=create_cnn(get_df, activation_function=afunc)

    
    get_df24 = read_stock(df, ticker= ticker, n_period= value, n_shift=-1)
    get_df24.dropna(how = "any", inplace = True)

    norm_test_data = get_df24.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]] #scale df_nextDay
    norm_test_data = np.expand_dims(norm_test_data, axis=-1)
    norm_test_target = get_df24["return"][:, None]


    score = [sqrt(metrics.mean_squared_error(cnn_model.predict(norm_test_data), norm_test_target)), metrics.r2_score(cnn_model.predict(norm_test_data), norm_test_target)]
    rf_result_list = [key,value,score[0],score[1]]
    
    eval_result_list = rf_result_list[:]

    # #-----------------------------------
    X_test, y_test = RF_crossval(df,n_periods = value) #manipulate data for deep learning
    X_test = np.expand_dims(X_test, axis=-1)
    predicted_y = cnn_model.predict(X_test) #prediction
    predicted_y = np.where(np.array(predicted_y) > 0, 1, 0)
    #print("Prediction results using Deep Learning is....")
    #print(metrics.mean_squared_error(y_test,predicted_y))

  
    # #add evaluation metrics to row
    
    #besides precision, recall, and fscore, append other metrics
    eval_result_list.append(metrics.mean_squared_error(y_test,predicted_y))
    eval_result_list.append(metrics.accuracy_score(y_test,predicted_y))
  
    # # #add row to data frame
    result = pd.DataFrame([eval_result_list],columns = results_col)
    results = results.append(result,ignore_index=True)
    # # #empty out eval_result_list
  return results

In [ ]:
cnn_table(dow_324, dow_323, afunc="relu" )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  if __name__ == '__main__':


Epoch 1/10
24/24 [==============================] - 0s 2ms/step - loss: 1.1047 - accuracy: 0.7064
Epoch 2/10
24/24 [==============================] - 0s 1ms/step - loss: 0.2450 - accuracy: 0.7099
Epoch 3/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 4/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 5/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 6/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 7/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 8/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 9/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accuracy: 0.7099
Epoch 10/10
24/24 [==============================] - 0s 1ms/step - loss: 4.1242e-08 - accu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  if __name__ == '__main__':


Epoch 1/10
24/24 [==============================] - 0s 1ms/step - loss: 0.5239 - accuracy: 0.1794
Epoch 2/10
24/24 [==============================] - 0s 2ms/step - loss: 1.0068e-04 - accuracy: 0.1815
Epoch 3/10
24/24 [==============================] - 0s 1ms/step - loss: 2.8386e-07 - accuracy: 0.1815
Epoch 4/10
24/24 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 0.1815
Epoch 5/10
24/24 [==============================] - 0s 2ms/step - loss: 2.8386e-07 - accuracy: 0.1815
Epoch 6/10
24/24 [==============================] - 0s 1ms/step - loss: 8.3891e-04 - accuracy: 0.1815
Epoch 7/10
24/24 [==============================] - 0s 2ms/step - loss: 2.8386e-07 - accuracy: 0.1815
Epoch 8/10
24/24 [==============================] - 0s 2ms/step - loss: 6.3347e-04 - accuracy: 0.1815
Epoch 9/10
24/24 [==============================] - 0s 2ms/step - loss: 2.8386e-07 - accuracy: 0.1815
Epoch 10/10
24/24 [==============================] - 0s 1ms/step - loss: 2.8386e-07 - accu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  if __name__ == '__main__':


Epoch 1/10
23/23 [==============================] - 0s 1ms/step - loss: 23.0059 - accuracy: 0.0000e+00
Epoch 2/10
23/23 [==============================] - 0s 1ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 3/10
23/23 [==============================] - 0s 971us/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 4/10
23/23 [==============================] - 0s 1ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 5/10
23/23 [==============================] - 0s 1ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 6/10
23/23 [==============================] - 0s 1ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 8/10
23/23 [==============================] - 0s 1ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 9/10
23/23 [==============================] - 0s 2ms/step - loss: 1.7176e-06 - accuracy: 0.0000e+00
Epoch 10/10
23/23 [============================

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.000371,0.0,0.226733,0.773267
1,1 minutes,6,0.000950,0.0,0.451849,0.548151
2,5 minutes,30,0.001694,0.0,0.582279,0.417721


## LSTM

In [ ]:

# Initialize the layers
def createSequentialModel(input_size, activation_function, num_hidden_layers, layer_neurons, dropout):   # softsign

    model = Sequential()
    model.add(LSTM(
        input_dim= input_size,
        units=5,
        return_sequences=True))
    model.add(Dropout(dropout))

    for i in range(num_hidden_layers):
                
        model.add(LSTM(layer_neurons, return_sequences=True))

        model.add(Dropout(dropout))

        model.add(Dense(units=1, activation = activation_function))

        print('Compiling model...')
        model.compile(loss='mse', optimizer='SGD')
    
    return model

In [ ]:
def lstm_table(df, df_P, ticker = 'WMT', afunc="relu", n_hl=1, n_neurons=20, dp=0.1):  # df_previousDay, df_nextDay,

  from sklearn import metrics
  from math import sqrt
  import sys



  period_list = {
    "10 s": 1,
    '1 minutes':6,
    '5 minutes':30
}

  results_col=["periods_name","periods","RMSE", "r2", 
              "mse", "accuracy_score"
            ]
  results = pd.DataFrame(columns = results_col)


  for key, value in period_list.items():
 
    get_df = read_stock(df_P, ticker= ticker, n_period= value, n_shift=-1)
    get_df.dropna(how = "any", inplace = True)
    norm_train_data = get_df.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]]
    norm_train_data = norm_train_data.to_numpy()
    norm_train_target = get_df["return"][:, None]
    model=createSequentialModel(norm_train_data.shape[1], activation_function=afunc, num_hidden_layers=n_hl, layer_neurons=n_neurons, dropout=dp)

    lstm_train_data= norm_train_data.reshape(norm_train_data.shape[0],1, norm_train_data.shape[1])   # 23 (2284, 1, 5)
    
    
    history= model.fit(
    lstm_train_data,
    norm_train_target,
    batch_size=100,
    epochs= 30,
    validation_split=0.2, shuffle = False)

    
    get_df24 = read_stock(df, ticker= ticker, n_period= value, n_shift=-1)
    get_df24.dropna(how = "any", inplace = True)

    norm_test_data = get_df24.loc[:,["VOI","OIR", "bid_ask_spread", "EWM", "OBV"]] #scale df_nextDay
    norm_test_data = norm_test_data.to_numpy()
    lstm_test_data= norm_test_data.reshape(norm_test_data.shape[0],1, norm_test_data.shape[1])    # (2099, 1, 5)
    norm_test_target = get_df24["return"][:, None]

    y_predict=model.predict(lstm_test_data)    # (2099, 1, 1)
    y_predict=y_predict.reshape((y_predict.shape[0], -1), order='F')   # (2099, 1)
    score = [sqrt(metrics.mean_squared_error(norm_test_target, y_predict)), metrics.r2_score(norm_test_target, y_predict)]
    rf_result_list = [key,value,score[0],score[1]]
    
    eval_result_list = rf_result_list[:]

    # # #-----------------------------------
    # X_test, y_test = RF_crossval(df,n_periods = value) #manipulate data for deep learning
    # X_test = X_test.to_numpy()
    #X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1])
    #predicted_y = model.predict(X_test) #prediction
    predicted_y = np.where(np.array(y_predict) > 0, 1, 0)
    norm_test_target = np.where(np.array(norm_test_target) > 0, 1, 0)
    #print("Prediction results using Deep Learning is....")
    #print(metrics.mean_squared_error(y_test,predicted_y))

  
    # #add evaluation metrics to row
    
    #besides precision, recall, and fscore, append other metrics
    eval_result_list.append(metrics.mean_squared_error(norm_test_target, predicted_y))
    eval_result_list.append(metrics.accuracy_score(norm_test_target, predicted_y))
  
    # # #add row to data frame
    result = pd.DataFrame([eval_result_list],columns = results_col)
    results = results.append(result,ignore_index=True)
    # # #empty out eval_result_list
  return results

In [ ]:
lstm_table(dow_324, dow_323, ticker = 'WMT', afunc="softsign", n_hl=1, n_neurons=20, dp=0.1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Compiling model...
Epoch 1/30
19/19 [==============================] - 1s 39ms/step - loss: 0.0034 - val_loss: 0.0021
Epoch 2/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0025 - val_loss: 0.0014
Epoch 3/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0020 - val_loss: 0.0011
Epoch 4/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 9.4935e-04
Epoch 5/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0017 - val_loss: 8.8350e-04
Epoch 6/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0016 - val_loss: 8.5154e-04
Epoch 7/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0017 - val_loss: 8.3186e-04
Epoch 8/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0016 - val_loss: 8.1745e-04
Epoch 9/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0016 - val_loss: 8.0226e-04
Epoch 10/30
19/19 [==============================] - 0s 3ms/step - loss: 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Compiling model...
Epoch 1/30
19/19 [==============================] - 1s 37ms/step - loss: 2.3522e-04 - val_loss: 2.1374e-04
Epoch 2/30
19/19 [==============================] - 0s 3ms/step - loss: 2.2892e-04 - val_loss: 1.9769e-04
Epoch 3/30
19/19 [==============================] - 0s 3ms/step - loss: 2.3376e-04 - val_loss: 1.9047e-04
Epoch 4/30
19/19 [==============================] - 0s 3ms/step - loss: 2.0755e-04 - val_loss: 1.8628e-04
Epoch 5/30
19/19 [==============================] - 0s 3ms/step - loss: 2.1715e-04 - val_loss: 1.8370e-04
Epoch 6/30
19/19 [==============================] - 0s 3ms/step - loss: 2.1179e-04 - val_loss: 1.8185e-04
Epoch 7/30
19/19 [==============================] - 0s 3ms/step - loss: 2.1140e-04 - val_loss: 1.8043e-04
Epoch 8/30
19/19 [==============================] - 0s 3ms/step - loss: 2.0310e-04 - val_loss: 1.7942e-04
Epoch 9/30
19/19 [==============================] - 0s 3ms/step - loss: 2.1243e-04 - val_loss: 1.7817e-04
Epoch 10/30
19/19 [=======

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


Compiling model...
Epoch 1/30
19/19 [==============================] - 1s 41ms/step - loss: 0.0041 - val_loss: 0.0030
Epoch 2/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 3/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0022 - val_loss: 0.0015
Epoch 4/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0018 - val_loss: 0.0012
Epoch 5/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0016 - val_loss: 0.0010
Epoch 6/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0014 - val_loss: 9.4833e-04
Epoch 7/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0014 - val_loss: 8.8045e-04
Epoch 8/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0013 - val_loss: 8.3506e-04
Epoch 9/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0013 - val_loss: 8.0088e-04
Epoch 10/30
19/19 [==============================] - 0s 3ms/step - loss: 0.0012 - 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.


,periods_name,periods,RMSE,r2,mse,accuracy_score
0,10 s,1,0.022109,-3544.055404,0.278195,0.721805
1,1 minutes,6,0.010870,-130.165185,0.429110,0.570890
2,5 minutes,30,0.022808,-190.608657,0.571224,0.428776
